# Big Data con Python

#### Rafael Caballero, Enrique Martín y Adrián Riesco. Editorial RC libros

## Capítulo 2:  ejemplo  de web scraping. Acceso a los datos del catastro


El propósito de este notebook es mostrar como hacer web scraping de datos de una página, en este ejemplo del catastro,  que nos exige introducir previamente datos e interaccionar, en este caso seleccionando pestañas y pulsando botones.

En particular utilizaremos la biblioteca *Selenium* que nos permite obtener automáticamente datos de páginas web dinámicas. Selenium está pensado realmente para automatizar pruebas de entornos web, pero a nosotros nos servirá para nuestro propósito de hacer web scraping.

En particular, lo que vamos a hacer es arrancar google Chrome y manejarlo automáticamente desde Python.



**Preparación**


Para ello necesitamos 3 cosas:

1.- Tener google Chrome

2.- Instalar la biblioteca selenium (pip install selenium)

3.- Tener un fichero controlador, un driver, de nombre *chromedriver.exe*. En nuestro caso supondremos que se encuentra en la carpeta c:/hlocal/tdm. El driver correspondiente a la versión de Chrome que tengamos se puede descargar de https://sites.google.com/a/chromium.org/chromedriver/downloads


#### Paso 1: abrir navegador
Comenzamos abriendo una sesión de Chrome de forma automática


**Errores comunes**

Es posible que al ejecutar este código obtengamos un error del tipo:
    
    SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 87. Current browser version is 90.0.4430.212 with binary path.....
    
 o similar. Esto indica que nuestra versión del driver no corresponde con la de nuestro navegador. La versión de Chrome, aunque nos la da el mensaje, la podemos consultar en el propio navegador, en *Configuración* (dentro del menú que se abre al hacer click sobre los 3 puntitos verticales, arriba a la derecha) + *Información de Chrome*.
 
 Con esta información iremos a  https://sites.google.com/a/chromium.org/chromedriver/downloads y descargaremos y descomprimiremos el fichero chromedriver correspondiente y ponemos el path correspondiente en la variable chromedriver
 
 
 Otro error que podemos obtener es alguno del tipo *FileNotFound* que se deberá, casi con seguridad a que debemos cambiar el path almacenado en la variable chromedriver del siguiente código


**Importante**
Una vez que logremos ques e abra el navegador no debemos teclear nada en él, ni cerrarlo el control lo llevaremos desde el programa en Python

In [6]:
# ejecutar esta casilla solo si queremos instalar o actualizar selenium; 
# muchas veces tras hacerlo deberemos hacer "Kernel->restart" para que los cambios se apliquen
import sys
!{sys.executable} -m pip install --upgrade --user selenium

Requirement already up-to-date: selenium in c:\users\rafa\appdata\roaming\python\python38\site-packages (4.4.3)


In [1]:
chromedriver = "./chromedriver.exe" # cambiar esta variable con el path a nuestro chromedriver
import os
from selenium import webdriver  # si da error, desde anaconda prompt hacer pip install --user  selenium
os.environ["webdriver.chrome.driver"] = chromedriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(executable_path=chromedriver,options=chrome_options)

<ipython-input-1-cfdeb6085767>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chromedriver,options=chrome_options)


### Paso 2: cargar página
Una vez abierto el navegador lo siguiente es cargar la página que deseemos, en este caso la del catastro. Debemos ver cómo se carga la página correspondiente

In [2]:
url = 'https://www1.sedecatastro.gob.es/CYCBienInmueble/OVCBusqueda.aspx'
driver.get(url)

### Paso 3, interacción con la página

Ahora queremos seleccionar la pestaña coordenadas, ya que vamos a introducir las coordenadas de un lugar concreto. Esto se hace en dos fases: primero se selecciona el elemento y luego interaccionamos con él. En este caso comprobamos viendo el código fuente que la etiqueta COORDENADAS es un enlace, sobre el que deseamos hacer click


In [3]:
from selenium.webdriver.common.by import By
coord   = driver.find_element(By.LINK_TEXT,"COORDENADAS")
coord.click()

Ya en la pestaña, introducimos las coordenadas y hacemos click para buscar la información. En este ejemplo hemos puesto las coordenadas  long=1.893817, lat=41.545639, pero pueden ser otras cualesquiera

In [4]:
lat = driver.find_element(By.ID,"ctl00_Contenido_txtLatitud")
lon = driver.find_element(By.ID,"ctl00_Contenido_txtLongitud")
latitud  = "41.545639 "
longitud = "1.893817"
lat.send_keys(latitud)
lon.send_keys(longitud)

datos = driver.find_element(By.ID,"ctl00_Contenido_btnDatos")
datos.click()

Ahora supongamos que queremos determinar el uso principal de este inmueble, para ellos buscamos 

In [6]:
xpath = "//*[./span/text()='Referencia catastral']//label"
etiqs = driver.find_element(By.XPATH,xpath)
print(etiqs.text)
xpath = "//*[./span/text()='Uso principal']//label"
etiqs = driver.find_element(By.XPATH,xpath)
print(etiqs.text)

7801701DF0070S0001QY    
Religioso


### Otras pruebas

Texto de la página

In [ ]:
html = driver.find_element(By.XPATH,"/html")
print(html.text)

Caminos absolutos

In [ ]:
head = driver.find_element(By.XPATH,"/html/head")
body = driver.find_element(By.XPATH,"/html/body")
html2 = body.find_element(By.XPATH,"/html")

Hijos de un elemento

In [ ]:
hijos = driver.find_elements(By.XPATH,"/html/body/*")
for element in hijos:
  print(element.tag_name)

Camino relativo

In [ ]:
divs = driver.find_elements(By.XPATH,"/html/body/*/div")
print(len(divs))

In [ ]:
divs = body.find_elements(By.XPATH,"./*/div")
print(len(divs))

Saltar pasos intermedios

In [ ]:
divs = driver.find_elements(By.XPATH,"/html/body//div")
print(len(divs))

In [ ]:
labels = driver.find_elements(By.XPATH,"//label")
print(len(labels))

Insertar texto

In [7]:
id = "ctl00_Contenido_tblInmueble"
div = driver.find_element(By.ID,id)
label = div.find_element(By.XPATH,"//label")
print(label.text)

7801701DF0070S0001QY    


In [ ]:
xpath = "//*[./span/text()='Referencia catastral']//label"
etiqs = driver.find_element(By.XPATH,xpath)
print(etiqs.text)

In [8]:
clase = driver.find_elements(By.XPATH,"(//label)[position()=3]")
print(clase[0].text)

etiqs = driver.find_elements(By.XPATH,"//label")
print(etiqs[2].text)

ulti = driver.find_elements(By.XPATH,"(//label)[last()]")
print(ulti[0].text)


In [9]:
driver.close()

Por Rafael Caballero. Del libro "Big data con Python"